# В этой домашке мы поработаем с эмбеддингами текстов и сделаем первое соревнование по нлп

## Начнем с подготовки (0.5 баллов)

Создадим "голову", которая будет отвечать за классификацию на основе эмбеддингов. Для этого обучим CatBoost на заранее подготовленных эмбеддингах с добавленным шумом.

In [1]:
import pandas as pd
 
train = pd.read_csv('FAKE_train.csv').drop(columns=['Unnamed: 0'])
test = pd.read_csv('FAKE_test.csv').drop(columns=['Unnamed: 0'])
train

,bert_dim_0,bert_dim_1,bert_dim_2,bert_dim_3,bert_dim_4,bert_dim_5,bert_dim_6,bert_dim_7,bert_dim_8,bert_dim_9,...,bert_dim_759,bert_dim_760,bert_dim_761,bert_dim_762,bert_dim_763,bert_dim_764,bert_dim_765,bert_dim_766,bert_dim_767,target
0,-0.045296,0.244643,0.146881,-0.101254,-0.354191,-0.580979,1.203313,0.624728,0.039920,-0.574070,...,-0.052892,0.023292,-0.154176,0.340931,-0.142372,-0.014048,-0.112122,0.264723,-0.180618,1
1,-0.833856,-0.186808,-0.461408,-0.482215,0.191647,0.173931,0.226668,0.462386,-0.625819,-0.180016,...,-0.069476,-0.059576,-0.754987,0.476386,-0.163974,-0.251988,-0.121008,0.281774,-0.028574,1
2,-0.553840,-0.868089,0.456105,-0.360056,0.596364,-0.175732,0.661811,0.303952,-0.157333,-0.344494,...,-0.411147,0.700571,-0.238418,-0.197669,-0.201720,-0.384008,0.014200,0.099776,0.380442,1
3,-0.709926,-0.059643,-0.065534,-0.215128,-0.341166,-0.002201,0.697558,0.685646,-0.670927,-0.089965,...,0.297650,0.686873,-0.404352,0.475008,0.230612,-0.003604,-0.798723,0.419383,0.010650,1
4,-0.127495,-0.026461,0.160941,0.114806,-0.279382,-0.752050,1.035819,0.550378,0.148431,-0.372095,...,0.057954,-0.508310,-0.334053,0.124174,0.029579,-0.142547,-0.883545,0.055623,0.213822,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7608,0.013373,0.053648,-0.172382,-0.199139,-0.172104,0.180905,0.537056,0.874261,-0.208165,-0.178274,...,-0.085405,-0.255601,-0.167974,0.482883,0.600995,-0.127468,-0.105035,0.021592,0.195847,1
7609,0.113711,0.180877,-0.105192,-0.032074,-0.497120,-0.442231,0.585394,0.361678,0.157206,-0.619585,...,-0.099414,-0.326833,-0.415066,0.137362,0.064457,-0.309373,-0.617580,0.308346,0.432512,1
7610,-0.218554,0.051987,0.221798,-0.167987,-0.171388,-0.090077,0.484971,0.561129,-0.152184,-0.275283,...,0.069041,-0.325151,-0.518002,0.083724,0.004744,0.034347,-0.159692,0.606469,0.353067,1
7611,-0.439928,-0.425846,0.083126,0.157701,-0.386743,0.026565,0.299643,0.243440,-0.287628,-0.245310,...,-0.009644,0.338429,0.078797,0.039647,-0.666714,0.179503,0.136507,0.158583,0.207595,1


In [2]:
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
import numpy as np

X = train.drop(columns = ['target'])
y = train['target']

# Сделайте разделение даты на трейн и валидацию с помощью train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [3]:
import torch


# Эту ячейку не изменяем до определенного этапа

class Head_catboost():
    def __init__(self,):
        # это основные параметры катбуста, которые пока что не изменяйте
        self.params = {
            'iterations': 1000,
            'learning_rate': 0.05,
            'depth': 6,
            'loss_function': 'Logloss',  # У нас бинарка
            'eval_metric': 'F1',  # Оптимизируем F1 как основной скор соревы
            'early_stopping_rounds': 150, 
            'random_seed': 42, # всегда фиксируйте сиды!
            'verbose': 100,
            'task_type': 'GPU' if torch.cuda.is_available() else 'CPU'
        } 
    
    def train(self, X_train, X_val, y_train, y_val):
        model = CatBoostClassifier(**self.params)
        model.fit(
            X_train, y_train,
            eval_set=(X_val, y_val),
            plot=True, # на кагле не заработает, но на личных машинках должен
            use_best_model=True # берется лучшая по скору на валидации
        )
        
        return model

In [4]:
model = Head_catboost()
model.train(X_train, X_val, y_train, y_val)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 0.6447312	test: 0.6125654	best: 0.6125654 (0)	total: 102ms	remaining: 1m 42s
100:	learn: 0.8004977	test: 0.7251064	best: 0.7308348 (93)	total: 4.39s	remaining: 39.1s
200:	learn: 0.8593527	test: 0.7443420	best: 0.7455919 (196)	total: 8.57s	remaining: 34.1s
300:	learn: 0.9058565	test: 0.7452282	best: 0.7487521 (295)	total: 12.7s	remaining: 29.6s
400:	learn: 0.9371181	test: 0.7447699	best: 0.7522936 (376)	total: 16.9s	remaining: 25.3s
500:	learn: 0.9599066	test: 0.7485380	best: 0.7522936 (376)	total: 21.1s	remaining: 21s
bestTest = 0.752293578
bestIteration = 376
Shrink model to first 377 iterations.


Теперь перейдём к получению сабмита. Условием соревнования является предсказание только классов — 0 или 1. Однако CatBoost по умолчанию возвращает вероятности.

Я предложу вам два варианта формирования сабмита. Подумайте (или проверьте на практике), какой из них окажется лучше, и попробуйте объяснить, почему один метод работает лучше другого.

In [16]:
from datetime import datetime
from torch.utils.data.dataloader import DataLoader
from transformers.trainer import DataCollatorWithPadding


def predict_argmax(model, tokenizer, test_df: pd.DataFrame, texts_column='text', batch_size=32, threshold=0.65):
    """
    Делает предсказания для модели BertForSequenceClassification и сохраняет сабмит.
    
    Args:
        model: обученная BertForSequenceClassification
        tokenizer: соответствующий токенизатор
        sample: DataFrame с колонкой текстов
        texts_column: имя колонки с текстами
        batch_size: размер батча для предсказаний
        threshold: порог для бинарного класса
    """
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.eval()

    texts = test_df[texts_column].tolist()
    predictions = []

    data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

    # создаём DataLoader для батчей
    class SimpleDataset(torch.utils.data.Dataset):
        def __init__(self, texts):
            self.texts = texts
        def __len__(self):
            return len(self.texts)
        def __getitem__(self, idx):
            return self.texts[idx]

    loader = DataLoader(SimpleDataset(texts), batch_size=batch_size, collate_fn=lambda batch: tokenizer(batch, return_tensors="pt", padding=True, truncation=True, max_length=512))

    # предсказание по батчам
    with torch.no_grad():
        for batch in loader:
            batch = {k: v.to(device) for k, v in batch.items()}
            logits = model(**batch).logits
            probs = torch.softmax(logits, dim=1)[:, 1]  # вероятность класса 1
            predictions.extend(probs.cpu().numpy())

    # применяем threshold
    predictions = (np.array(predictions) >= threshold).astype(int)

    submission_df = pd.DataFrame({
        'id': test_df['id'],
        'target': predictions
    })
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    submission_df.to_csv(f'Submission_{timestamp}.csv', index=False)


(сабмиты можно найти во вкладке Output справа в меню кагла, после чего загрузить в соревнование)

Пишите ваш ответ тут: 

# Теперь будем создавать эмбеддинги самостоятельно

Вам предстоит попробовать несколько подходов к построению текстовых представлений.

Для каждого текста в трейне создайте эмбеддинг, после чего обучите CatBoost и посмотрите  полученный скор в соревновании. Мы посмотрим, какой метод покажет себя лучше всего.

In [3]:
import pandas as pd


train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
test_df = pd.read_csv('sample_submission.csv')
train

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1


In [5]:
from sklearn.model_selection import train_test_split



X = train.drop(columns=['target'])
y = train['target']

# Сделайте разделение даты на трейн и валидацию с помощью train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

## 1. Bag-of-Words (BOW) (0.75 балла)

Метод создаёт числовое представление текста, где каждое слово превращается в отдельную колонку, а значение указывает частоту его встречаемости. Такой подход полностью игнорирует порядок слов и их смысловые связи, опираясь только на факт их наличия в тексте.

In [8]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()

X_BOW_train = vectorizer.fit_transform(X_train['text'])
X_BOW_val = vectorizer.transform(X_val['text'])

In [9]:
model = Head_catboost()
model.train(X_BOW_train, X_BOW_val, y_train, y_val)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 0.6263319	test: 0.6112327	best: 0.6112327 (0)	total: 32ms	remaining: 32s
100:	learn: 0.6947935	test: 0.6732673	best: 0.6732673 (100)	total: 3.08s	remaining: 27.5s
200:	learn: 0.7404014	test: 0.6993865	best: 0.6993865 (200)	total: 5.99s	remaining: 23.8s
300:	learn: 0.7682376	test: 0.7162630	best: 0.7173725 (294)	total: 8.82s	remaining: 20.5s
400:	learn: 0.7875936	test: 0.7186147	best: 0.7217916 (326)	total: 11.6s	remaining: 17.3s
500:	learn: 0.7976165	test: 0.7256944	best: 0.7260156 (480)	total: 14.4s	remaining: 14.3s
600:	learn: 0.8065756	test: 0.7299652	best: 0.7299652 (586)	total: 17.2s	remaining: 11.4s
700:	learn: 0.8154011	test: 0.7328111	best: 0.7376623 (691)	total: 19.8s	remaining: 8.46s
800:	learn: 0.8270869	test: 0.7312391	best: 0.7376623 (691)	total: 22.5s	remaining: 5.6s
bestTest = 0.7376623377
bestIteration = 691
Shrink model to first 692 iterations.


**Полученный скор:**

## 2. TF-IDF (0.75 балла)

Учитывает не только частоту слова в документе (TF), но и его редкость во всей коллекции документов (IDF), автоматически понижая вес распространенных слов. Это позволяет выделять действительно значимые термины, которые характерны для конкретного текста.

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer()

X_tfidf_train = tfidf_vectorizer.fit_transform(X_train['text'])
X_tfidf_val = tfidf_vectorizer.transform(X_val['text'])

model = Head_catboost()
model.train(X_tfidf_train, X_tfidf_val, y_train, y_val)
#ваш выбранный метод

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 0.6231563	test: 0.6178862	best: 0.6178862 (0)	total: 43.2ms	remaining: 43.2s
100:	learn: 0.7004939	test: 0.6821429	best: 0.6821429 (100)	total: 4.01s	remaining: 35.7s
200:	learn: 0.7492355	test: 0.6994819	best: 0.7038328 (180)	total: 7.78s	remaining: 30.9s
300:	learn: 0.7799527	test: 0.7103918	best: 0.7109974 (286)	total: 11.4s	remaining: 26.4s
400:	learn: 0.8005115	test: 0.7250000	best: 0.7250000 (398)	total: 14.9s	remaining: 22.2s
500:	learn: 0.8124603	test: 0.7300000	best: 0.7300000 (497)	total: 18.4s	remaining: 18.4s
600:	learn: 0.8216399	test: 0.7283333	best: 0.7321131 (588)	total: 21.9s	remaining: 14.6s
700:	learn: 0.8310483	test: 0.7312187	best: 0.7321131 (588)	total: 25.4s	remaining: 10.9s
bestTest = 0.7321131448
bestIteration = 588
Shrink model to first 589 iterations.


**Полученный скор:**

## 3. Word2Vec (0.75 балла)

Нейронная сеть преобразует слова в векторы фиксированной длины так, что семантически близкие слова оказываются рядом в пространстве. Обучение идёт либо в режиме skip-gram (слово → контекст), либо CBOW (контекст → слово), что позволяет сохранять смысловые связи.

In [12]:
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer


X_train_tokens = X_train["text"].str.lower().apply(word_tokenize)
X_val_tokens = X_val["text"].str.lower().apply(word_tokenize)

corpus = X_train_tokens.tolist()


w2v = Word2Vec(
    sentences=corpus,
    vector_size=100,
    window=5,
    sg=1,
    min_count=1,
    workers=4
)

def text_to_vector(tokens, model, vector_size=100):
    vectors = [model.wv[w] for w in tokens if w in model.wv]
    if len(vectors) == 0:
        return np.zeros(vector_size)  # если слова нет в словаре
    return np.mean(vectors, axis=0)

X_w2v_train = np.vstack(X_train_tokens.apply(lambda x: text_to_vector(x, w2v)))
X_w2v_val =  np.vstack(X_val_tokens.apply(lambda x: text_to_vector(x, w2v)))
model = Head_catboost()
model.train(X_w2v_train, X_w2v_val, y_train, y_val)
#ваш выбранный метод

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 0.6186891	test: 0.6243902	best: 0.6243902 (0)	total: 32.4ms	remaining: 32.4s
100:	learn: 0.6903775	test: 0.6589975	best: 0.6606409 (90)	total: 2.7s	remaining: 24s
200:	learn: 0.7199309	test: 0.6688259	best: 0.6709992 (175)	total: 5.24s	remaining: 20.8s
300:	learn: 0.7468597	test: 0.6693614	best: 0.6747377 (222)	total: 7.63s	remaining: 17.7s
400:	learn: 0.7623888	test: 0.6783719	best: 0.6836653 (382)	total: 9.96s	remaining: 14.9s
500:	learn: 0.7752857	test: 0.6767276	best: 0.6836653 (382)	total: 12.3s	remaining: 12.3s
bestTest = 0.6836653386
bestIteration = 382
Shrink model to first 383 iterations.


**Полученный скор:**

## 4. GloVe (0.75 балла)

Строит векторные представления слов на основе статистики их совместной встречаемости во всем корпусе текстов, вычисляя, насколько часто пары слов появляются вместе. Оптимизирует вектора так, чтобы их скалярное произведение соответствовало логарифму вероятности совместного появления слов.

In [22]:
# === 1. Загружаем предобученные эмбеддинги Glove ===
def load_glove(path, dim=300):
    embeddings = {}
    with open(path, encoding="utf8") as f:
        for line in f:
            parts = line.rstrip().split()
            if len(parts) <= dim:  # если меньше, чем слово + dim чисел → битая строка
                continue
            word = " ".join(parts[:-dim])   # всё, что до последних dim токенов
            try:
                vector = np.asarray(parts[-dim:], dtype="float32")
            except ValueError:
                continue
            if vector.shape[0] != dim:
                continue
            embeddings[word] = vector
    return embeddings


glove = load_glove("wiki_giga_2024_300_MFT20_vectors_seed_2024_alpha_0.75_eta_0.05_combined.txt")  # скачай заранее с https://nlp.stanford.edu/projects/glove/

# === 2. Функция: превращаем текст в средний вектор ===
def text_to_vector(text, embeddings, dim=300):
    words = text.split()
    vecs = [embeddings[w] for w in words if w in embeddings]
    if len(vecs) == 0:
        return np.zeros(dim)
    return np.mean(vecs, axis=0)


X_glove_train = np.vstack(X_train["text"].apply(lambda x: text_to_vector(str(x), glove, dim=300)))
X_glove_val = np.vstack(X_val["text"].apply(lambda x: text_to_vector(str(x), glove, dim=300)))


In [23]:
model = Head_catboost()
model.train(X_glove_train, X_glove_val, y_train, y_val)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 0.6423510	test: 0.6361366	best: 0.6361366 (0)	total: 31.4ms	remaining: 31.4s
100:	learn: 0.7403291	test: 0.6776145	best: 0.6798965 (89)	total: 3.26s	remaining: 29s
200:	learn: 0.7864407	test: 0.6770563	best: 0.6827171 (113)	total: 6.41s	remaining: 25.5s
300:	learn: 0.8148771	test: 0.6804124	best: 0.6827586 (250)	total: 9.57s	remaining: 22.2s
400:	learn: 0.8373066	test: 0.6791809	best: 0.6860565 (352)	total: 12.6s	remaining: 18.8s
500:	learn: 0.8535270	test: 0.6752137	best: 0.6860565 (352)	total: 15.6s	remaining: 15.6s
bestTest = 0.6860564585
bestIteration = 352
Shrink model to first 353 iterations.


**Полученный скор:**

## 5. Трансформерный подход (0.75 балла)

Для работы с NLP моделями используйте сайт https://huggingface.co/ и их библиотеку transformers. Сначала будем использовать базовую версию BERT https://huggingface.co/google-bert/bert-base-uncased 

По желанию можете еще сделать вариант с использованием Pipeline

In [10]:
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn as nn
import numpy as np
from tqdm import tqdm


class Bert(nn.Module):
    def __init__(self, trainable=False):
        super().__init__()
        self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        self.bert = AutoModel.from_pretrained('bert-base-uncased').to(self.device) 
        self.tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

        self.CLS = 0

    def forward(self, input):
        # TO DO
        """
        Получаем текстовый вход, токенизирует его и пропускает через BERT-модель,
        возвращая векторное представление текста (CLS-токен).
        
        Args:
            input_text (str): Входной текст
            
        Returns:
            torch.Tensor: Векторное представление [1, hidden_size]
        """
        self.bert.eval()  # если не тренируем BERT
        with torch.no_grad():  # отключаем градиенты для ускорения
            encoded = self.tokenizer(
                input,
                return_tensors="pt",
                padding=True,
                truncation=True,
                max_length=512
            ).to(self.device)

            out = self.bert(**encoded)  # передаем как kwargs
            cls_vec = out.last_hidden_state[:, self.CLS, :]  # [batch_size, hidden_size]
            
        return cls_vec.cpu().numpy()


In [21]:
base_bert = Bert()


X_bert_train = np.vstack(X_train["text"].apply(lambda x: base_bert(x)))
X_bert_val = np.vstack(X_val["text"].apply(lambda x: base_bert(x)))

In [22]:
model = Head_catboost()
model.train(X_bert_train, X_bert_val, y_train, y_val)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 0.6698907	test: 0.6605042	best: 0.6605042 (0)	total: 58.8ms	remaining: 58.7s
100:	learn: 0.8125384	test: 0.7545910	best: 0.7545910 (100)	total: 4.21s	remaining: 37.5s
200:	learn: 0.8628967	test: 0.7615894	best: 0.7677100 (187)	total: 8.41s	remaining: 33.4s
300:	learn: 0.8956121	test: 0.7723911	best: 0.7727646 (299)	total: 12.4s	remaining: 28.8s
400:	learn: 0.9218564	test: 0.7725041	best: 0.7750411 (306)	total: 16.4s	remaining: 24.5s
bestTest = 0.7750410509
bestIteration = 306
Shrink model to first 307 iterations.


## 6. Fine-Tuning (2.75 баллов)

Теперь сделаем файнтюнинг BERT для нашей задачи, добавим линейный слой на CLS токен и дообучим полученную модель на наши данные. Файнтюнинг самый эффективный метод, если у вас есть ресурсы и время для обучения.

Как я показывал на семенаре можно обучать с помощью torch или с помощью Trainer из библиотеки transformers. 
Реализуйте оба варианта. Но в коротких соревнованиях отдавайте предпочтение Trainer

В нашем случае мы можем воспринимать задачу как регрессию так и классификацию, это будет зависить от выбранного вами подхода обучения линейного слоя на CLS токене - линейный слой размерностью (d_cls_Berta, 1) - регрессия или же (d_cls_Berta, 2) - классификация. Лично я использовал бы классификацию и обучал бы на CrossEntropyLoss. Пока что реализуйте эту идею, потом можете поэксперементировать.

### HF Trainer

In [11]:
from sklearn.metrics import f1_score
from transformers import (
    BertTokenizer, 
    BertForSequenceClassification,
    Trainer, 
    TrainingArguments,
    DataCollatorWithPadding
)

model = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased',
    num_labels=2,  # Количество классов
)



def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = logits.argmax(axis=-1)
    f1 = f1_score(labels, preds)
    return {'f1': f1}


# BertForSequenceClassification автоматически добавляет линейный слой на клс модели с выбранным количеством лейблов
#TO DO
'''
Обучите берт используя Trainer, после чего создайте сабмит с его предсказаниеями и проверьте результат на кагле
'''

# используйте эти параметры обучения
training_args = TrainingArguments(
    # Основные параметры
    output_dir='./bert-binary-classifier',  # Директория для сохранения
    
    # Параметры обучения
    num_train_epochs=3,                     # Количество эпох
    per_device_train_batch_size=64,         # Размер батча для обучения
    per_device_eval_batch_size=64,          # Размер батча для валидации
    learning_rate=2e-5,                     # Learning rate
    warmup_ratio = 0.1,                     # 10% от общего числа шагов для вармапа или warmup_steps = int(0.1 * total_training_steps)
    lr_scheduler_type = 'cosine',           # Можете посмотреть на них в 
                                            # https://www.kaggle.com/code/snnclsr/learning-rate-schedulers 
                                            # соответсвующий ему будет get_cosine_schedule_with_warmup
    
    # Сохранение и логирование
    logging_dir='./logs',                   # Директория для логов
    logging_steps=100,                      # Частота логирования
    save_steps=500,                         # Частота сохранения
    save_total_limit=2,                     # Максимум чекпоинтов
    save_strategy='epoch',                  # Стратегия сохранения
    
    # Валидация
    eval_strategy='epoch',            # Стратегия валидации
    eval_steps=500,                         # Шаги для валидации
    load_best_model_at_end=True,            # Загружать лучшую модель
    metric_for_best_model='f1',             # Метрика для выбора лучшей
    greater_is_better=True,                 # Больше значение = лучше

    # воспроизводимость
    seed=42,                                # Seed для воспроизводимости
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### Torch

In [7]:
import torch


class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
        
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [12]:
X_train = X_train.reset_index(drop=True)
X_val = X_val.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)
y_val = y_val.reset_index(drop=True)

tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

train_encodings = tokenizer(
    X_train['text'].tolist(), 
    padding=True, 
    truncation=True
)
val_encodings = tokenizer(
    X_val['text'].tolist(), 
    padding=True, 
    truncation=True
)
test_encodings = tokenizer(
    test['text'].tolist(), 
    padding=True, 
    truncation=True
)
train_dataset = CustomDataset(train_encodings, y_train)
val_dataset = CustomDataset(val_encodings, y_val)

In [13]:
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn as nn
import numpy as np
from tqdm import tqdm

#TO DO
'''
Обучите берт используя Torch используйте параметры обучения из ячейки выше,
после чего создайте сабмит с его предсказаниеями и проверьте результат на кагле
'''


def train():
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        compute_metrics=compute_metrics
    )

    trainer.train()

train()

Epoch,Training Loss,Validation Loss,F1
1,No log,0.418596,0.787365
2,0.536600,0.419562,0.799090
3,0.369300,0.408295,0.806880


In [ ]:
def predict(model, tokenizer, test_df: pd.DataFrame, texts_column='text', batch_size=32, threshold=0.65):
    """
    Делает предсказания для модели BertForSequenceClassification и сохраняет сабмит.
    
    Args:
        model: обученная BertForSequenceClassification
        tokenizer: соответствующий токенизатор
        sample: DataFrame с колонкой текстов
        texts_column: имя колонки с текстами
        batch_size: размер батча для предсказаний
        threshold: порог для бинарного класса
    """
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.eval()

    texts = test_df[texts_column].tolist()
    predictions = []

    data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

    # создаём DataLoader для батчей
    class SimpleDataset(torch.utils.data.Dataset):
        def __init__(self, texts):
            self.texts = texts
        def __len__(self):
            return len(self.texts)
        def __getitem__(self, idx):
            return self.texts[idx]

    loader = DataLoader(SimpleDataset(texts), batch_size=batch_size, collate_fn=lambda batch: tokenizer(batch, return_tensors="pt", padding=True, truncation=True, max_length=512))

    # предсказание по батчам
    with torch.no_grad():
        for batch in loader:
            batch = {k: v.to(device) for k, v in batch.items()}
            logits = model(**batch).logits
            probs = torch.softmax(logits, dim=1)[:, 1]  # вероятность класса 1
            predictions.extend(probs.cpu().numpy())
    return predictions

In [17]:
preds = predict(model, tokenizer, test)

In [37]:
predict_argmax(model, AutoTokenizer.from_pretrained('bert-base-uncased'), test)

# Для самых пытливых, которые хотят участвовать и выигрывать в соревнованиях (3 балла)

Теперь я даю вам полную свободу в эксперементах, записывайте логи и идеи ниже, попробуйте занять хорошее место в этой соревке. 

Первым делом я бы предложил поперебирать модели. Для удобства в подборе можно использовать AutoModelForSequenceClassification. 

Для выбора моделей воспользуйтесь рейтингом MTEB - https://huggingface.co/spaces/mteb/leaderboard (новая версия) / https://huggingface.co/spaces/mteb/leaderboard_legacy (легаси версия)

Можете еще посмотреть библиотеку SentenceTransformer (является оберткой для Transformers от сбера), но я ей не то чтобы много пользовался, также у нее куча проблем в плане совместимостей и на всеросе она мне куду положила)))

In [ ]:
from sentence_transformers import (
    SentenceTransformer, SentenceTransformerTrainer,
    SentenceTransformerTrainingArguments
)
from sentence_transformers.losses import MultipleNegativesRankingLoss
from datasets import load_dataset

model = SentenceTransformer("Qwen/Qwen3-Embedding-8B")

model.safetensors.index.json: 0.00B [00:00, ?B/s]

c:\ProgramData\anaconda3\envs\NLP\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Степан\.cache\huggingface\hub\models--Qwen--Qwen3-Embedding-8B. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.90G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/336M [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

Оценка баллов за этот пункт будет делиться на две части, 

- качество проведенных эксперементов максимум один балл 

- баллы за соревнование 2 * (1 - (Ваше_место - 1)/(количество_участников - 1))

**Описание экспериментов**
- Обучаем Qwen3-Embedding-0.6B (взят с ЛБ) с такими же параметрами. Результат 0.82
- Обучаем Qwen3-Embedding-0.6B. Предварительно делаем ей более корректную предсказывающую голову: Linear - Relu - Dropout - Linear. Отдельно обучаем предсказывающую голову с большим LR, затем всю модель вместе с маленьким LR (1e-5). Результат 0.81
- Достаем эмбеддинги из обученной Qwen3-Embedding-0.6B и используем их как вход в CatBoost. также используем колонки keyword и location, данные нам изначально. Результат 0.83
- Усредняем предсказания прошлого эксперимента с предсказаниями BERTа из baseline. Результат 0.84